<a href="https://colab.research.google.com/github/Ali-MH-Mansour/Document-classification/blob/main/CodeForPaper/Document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SBERT

In [1]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('all-mpnet-base-v2')
#multi-qa-mpnet-base-dot-v1
#all-mpnet-base-v2

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
import numpy as np
from scipy.spatial.distance import cosine
# Set the maximum length for each document part
max_part_length = 500

def process_documents_sbert(documents):
    """
    Processes a collection of documents and generates embeddings for each part.

    Args:
        documents (list): A list of document strings.

    Returns:
        dict: A dictionary where the keys are the document IDs and the values are document embeddings.
    """
    document_embeddings = {}
    document_embeddings_first = {}
    for doc_id, document in enumerate(documents):
        # Split the document into parts if it's longer than the maximum length
        parts = [document[i:i+max_part_length] for i in range(0, len(document), max_part_length)]
        if len(parts)> 1 and len(parts[-1])<400:
            parts = parts[:-1]
        # Generate embeddings for each part
        part_embeddings = []

        for part in parts:
            vector = bert_model.encode(part)
            part_embeddings.append(vector)

        # Calculate the average of the part embeddings as the document embedding
        document_embedding = np.mean(part_embeddings, axis=0)

        # Store the document embedding
        document_embeddings[doc_id] = document_embedding
        document_embeddings_first[doc_id] = part_embeddings[0] # all

    return document_embeddings, document_embeddings_first





def get_similarity(vector1, vector2):
    """
    Calculates the cosine similarity between two BERT vectors.

    Args:
        vector1 (numpy.ndarray): The first BERT vector.
        vector2 (numpy.ndarray): The second BERT vector.

    Returns:
        float: The cosine similarity between the two vectors, ranging from -1 to 1.
    """
    similarity = 1 - cosine(vector1, vector2)
    return similarity

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

In [11]:
BBC_file_path="/content/drive/MyDrive/PHD/Data/bbc-text.csv"

BBC_raw_path="/content/drive/MyDrive/PHD/Data/bbc_raw.csv"

OH = "/content/drive/MyDrive/PHD/Data/ohsumed-allcats.csv"
Reuters_file_path = "/content/drive/MyDrive/PHD/Data/ReuterKaggle.csv"
oldRE= "/content/drive/MyDrive/PHD/Data/Reuters88.csv"
rrr = "/content/drive/MyDrive/PHD/Data/RE2part/DataTestDF.csv"
rest= "/content/drive/MyDrive/PHD/Data/RE2part/DataTrainDF.csv"
company = "/content/drive/MyDrive/PHD/Keyword/All_companies_profiles.xlsx"
# data_DF = pd.read_excel(company)
# data_DF = data_DF[['description', 'CompanyName']]
# data_DF.head(5)


data_DF = pd.read_csv(BBC_file_path)

data_DF.columns.values[0] = "class"
data_DF.columns.values[1] = "document"

data_DF['class'] = pd.factorize(data_DF['class'])[0]
data_DF.head()

,class,document
0,0,tv future in the hands of viewers with home th...
1,1,worldcom boss left books alone former worldc...
2,2,tigers wary of farrell gamble leicester say ...
3,2,yeading face newcastle in fa cup premiership s...
4,3,ocean s twelve raids box office ocean s twelve...


In [27]:
%%time
docs = data_DF['document'].tolist()
doc2vec_embeddings, embeddings_first  = process_documents_sbert(docs)

CPU times: user 1h 29min 26s, sys: 8.56 s, total: 1h 29min 34s
Wall time: 1h 30min 33s


In [6]:
import pickle

data_path = "/content/drive/MyDrive/Research/avg_embeddings_bbc2.pkl"

with open(data_path,'rb') as fin:
    doc2vec_embeddings = pickle.load(fin)

data_path2 = "/content/drive/MyDrive/Research/first_embeddings_bbc2.pkl"

with open(data_path2,'rb') as fin:
    doc2vec_embeddings2 = pickle.load(fin)



In [19]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.metrics import f1_score

#-------------------------------------------------------------------------------
classes = data_DF['class'].tolist()
x_train , x_test , y_train , y_test = train_test_split(doc2vec_embeddings2 , classes , test_size = 0.30 , random_state = 0)

kr = ["poly", "rbf","linear"]
for k in kr:

  svm =  SVC(kernel=k, random_state=44).fit(x_train , y_train)
  y_pred=svm.predict(x_test)
  F1 = f1_score(y_test,y_pred, average="micro")
  print("The micro average-weighted f1 score = ", F1, " Kernel ", k)

The micro average-weighted f1 score =  0.9760479041916168  Kernel  poly
The micro average-weighted f1 score =  0.9805389221556886  Kernel  rbf
The micro average-weighted f1 score =  0.9730538922155688  Kernel  linear
CPU times: user 3.49 s, sys: 5.84 ms, total: 3.49 s
Wall time: 3.8 s


In [28]:
import pickle
with open("/content/drive/MyDrive/Research/avg_embeddings_oh.pkl", "wb") as fOut:
    pickle.dump(doc2vec_embeddings,fOut)

In [29]:
with open("/content/drive/MyDrive/Research/first_embeddings_oh2.pkl", "wb") as fOut:
    pickle.dump(embeddings_first ,fOut)

# Split data

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

#-------------------------------------------------------------------------------
classes = data_DF['class'].tolist()
x_train , x_test , y_train , y_test = train_test_split(doc2vec_embeddings2 , classes , test_size = 0.30 , random_state = 0)

## Neural net classifier

In [37]:
%%time
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=500).fit(x_train , y_train)
y_pred=clf.predict(x_test)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.9790419161676647
CPU times: user 17.2 s, sys: 11.8 s, total: 28.9 s
Wall time: 24.2 s


In [38]:
%%time
# for tfidf change Xvectors, Xtest
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5).fit(x_train , y_train)


y_pred= neigh.predict(x_test)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.9416167664670658
CPU times: user 26.9 s, sys: 12.2 ms, total: 26.9 s
Wall time: 35.7 s


# TFIDF

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

labels = data_DF['class'].tolist()
x_train , x_test , y_train , y_test = train_test_split(data_DF['document'] , classes , test_size = 0.30 , random_state = 0)
vectorizer = TfidfVectorizer(use_idf=True, max_df = 0.5,min_df = 10, ngram_range=(1,1))

In [32]:
Xvectors = vectorizer.fit_transform(x_train)

In [33]:
X_test = vectorizer.transform(x_test)

kr = ["poly","linear", "rbf"]
for k in kr:

  svm =  SVC(kernel=k, random_state=44).fit(Xvectors , y_train)
  y_pred=svm.predict(X_test)
  F1 = f1_score(y_test,y_pred, average="micro")
  print("The micro average-weighted f1 score = ", F1, " Kernel ", k)

The micro average-weighted f1 score =  0.874251497005988  Kernel  poly
The micro average-weighted f1 score =  0.9775449101796407  Kernel  linear
The micro average-weighted f1 score =  0.9820359281437125  Kernel  rbf


In [39]:
%%time
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=500).fit(Xvectors , y_train)
y_pred=clf.predict(X_test)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

In [40]:
%%time
# for tfidf change Xvectors, Xtest
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5).fit(Xvectors , y_train)


y_pred= neigh.predict(X_test)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

# LSA

In [41]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [42]:
svd = TruncatedSVD(300)
lsa = make_pipeline(svd, Normalizer(copy=False))

In [43]:
# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(Xvectors)

In [44]:
X_test_lsa = lsa.transform(X_test)

In [45]:
kr = ["poly","linear", "rbf"]
for k in kr:

  svm =  SVC(kernel=k, random_state=44).fit(X_train_lsa , y_train)
  y_pred=svm.predict(X_test_lsa)
  F1 = f1_score(y_test,y_pred, average="micro")
  print("The micro average-weighted f1 score = ", F1, " Kernel ", k)

The micro average-weighted f1 score =  0.9730538922155688  Kernel  poly
The micro average-weighted f1 score =  0.9775449101796407  Kernel  linear
The micro average-weighted f1 score =  0.9760479041916168  Kernel  rbf


In [47]:
%%time
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=500).fit(X_train_lsa , y_train)
y_pred=clf.predict(X_test_lsa)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.9820359281437125
CPU times: user 5.16 s, sys: 2.33 s, total: 7.49 s
Wall time: 3.99 s


In [48]:
%%time
# for tfidf change Xvectors, Xtest
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5).fit(X_train_lsa , y_train)


y_pred= neigh.predict(X_test_lsa)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.9461077844311377
CPU times: user 107 ms, sys: 1.17 ms, total: 108 ms
Wall time: 134 ms
